# Installation

In [1]:
!pip install cirq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 2.9 MB/s 
     |████████████████████████████████| 66 kB 4.4 MB/s 
     |████████████████████████████████| 594 kB 36.6 MB/s 
     |████████████████████████████████| 1.8 MB 48.9 MB/s 
     |████████████████████████████████| 576 kB 43.2 MB/s 
     |████████████████████████████████| 120 kB 54.1 MB/s 
     |████████████████████████████████| 221 kB 57.6 MB/s 
     |████████████████████████████████| 44 kB 2.6 MB/s 
     |████████████████████████████████| 147 kB 63.6 MB/s 
     |████████████████████████████████| 229 kB 70.0 MB/s 
     |████████████████████████████████| 84 kB 3.9 MB/s 
     |████████████████████████████████| 49 kB 5.7 MB/s 
     |████████████████████████████████| 68 kB 6.8 MB/s 
     |████████████████████████████████| 58 kB 6.2 MB/s 
     |████████████████████████████████| 80 kB 8.8 MB/s 
     |████████████████████████████████| 119 kB 

# Import Packages

In [2]:
import cirq
import numpy as np
import sympy
import math

# Hamiltonian Simulation

In [10]:
class HamiltonianSimulation(cirq.EigenGate, cirq.testing.SingleQubitGate):
    """
    Class này sẽ tạo ma trận đơn nhất từ một ma trận Hermitian đầu vào _H_ và
    thời gian t.
    """

    def __init__(self, _H_, t, exponent=1.0):
        cirq.testing.SingleQubitGate.__init__(self)
        cirq.EigenGate.__init__(self, exponent=exponent)
        self._H_ = _H_
        self.t = t
        eigen_vals, eigen_vecs = np.linalg.eigh(self._H_)
        self.eigen_components = []
        for _lambda_, vec in zip(eigen_vals, eigen_vecs.T):
            theta = -_lambda_*t / np.pi
            _proj_ = np.outer(vec, np.conj(vec))
            self.eigen_components.append((theta, _proj_))

    def _with_exponent(self, exponent):
        return HamiltonianSimulation(self._H_, self.t, exponent)

    def _eigen_components(self):
        return self.eigen_components

# Quantum Phase Estimation

In [11]:
class QFT:
    """
    Quantum Fourier Transform
    Mình lấy lại code của bài 3: Mọi xem rõ hơn ở https://github.com/qmlvietnam/CodeforBlog/blob/main/QFT.ipynb
    """

    def __init__(self, signal_length=16,
                 basis_to_transform='',
                 validate_inverse_fourier=False,
                 qubits=None):
        
        self.signal_length = signal_length
        self.basis_to_transform = basis_to_transform
        
        if qubits is None:
            self.num_qubits = int(np.log2(signal_length))
            self.qubits = [cirq.LineQubit(i) for i in range(self.num_qubits)]
        else:
            self.qubits = qubits
            self.num_qubits = len(self.qubits)

        self.qubit_index = 0
        self.input_circuit = cirq.Circuit()

        self.validate_inverse_fourier = validate_inverse_fourier
        self.circuit = cirq.Circuit()
        # if self.validate_inverse_fourier:
        self.inv_circuit = cirq.Circuit()

        for k, q_s in enumerate(self.basis_to_transform):
            if int(q_s) == 1:
                # Change the qubit state from 0 to 1 
                self.input_circuit.append(cirq.X(self.qubits[k]))

    def qft_circuit_iter(self):

        if self.qubit_index > 0:
            # Apply the rotations on the prior qubits
            # conditioned on the current qubit
            for j in range(self.qubit_index):
                diff = self.qubit_index - j + 1
                rotation_to_apply = -2.0 / (2.0 ** diff)
                self.circuit.append(cirq.CZ(self.qubits[self.qubit_index],
                                            self.qubits[j]) ** rotation_to_apply)
        # Apply the Hadamard Transform 
        # on current qubit
        self.circuit.append(cirq.H(self.qubits[self.qubit_index]))
        # set up the processing for next qubit
        self.qubit_index += 1

    def qft_circuit(self):

        while self.qubit_index < self.num_qubits:
            self.qft_circuit_iter()
            # See the progression of the Circuit built
            print(f"Circuit after processing Qubit: {self.qubit_index - 1} ")
            print(self.circuit)
        # Swap the qubits to match qft definititon
        self.swap_qubits()
        print("Circuit after qubit state swap:")
        print(self.circuit)
        # Create the inverse Fourier Transform Circuit
        self.inv_circuit = cirq.inverse(self.circuit.copy())

    def swap_qubits(self):
        for i in range(self.num_qubits // 2):
            self.circuit.append(cirq.SWAP(self.qubits[i], self.qubits[self.num_qubits - i - 1]))

    def simulate_circuit(self):
        sim = cirq.Simulator()
        result = sim.simulate(self.circuit)
        return result

In [12]:
class ControlledUnitary(cirq.Gate):

    def __init__(self, num_qubits, num_input_qubits, U):
        self._num_qubits = num_qubits
        self.num_input_qubits = num_input_qubits
        self.num_control_qubits = num_qubits - self.num_input_qubits
        self.U = U

    def num_qubits(self) -> int:
        return self._num_qubits

    def _decompose_(self, qubits):
        qubits = list(qubits)
        input_state_qubit = qubits[:self.num_input_qubits]
        control_qubits = qubits[self.num_input_qubits:]

        for i,q in enumerate(control_qubits):
            _pow_ = 2 ** (self.num_control_qubits - i - 1)
            #yield self.U(q, *input_state_qubit)**_pow_
            yield cirq.ControlledGate(self.U**_pow_)(q, *input_state_qubit)
            


class QuantumPhaseEstimation:
    '''
    Class QuantumPhaseEstimation được cải tiến từ code bài 4 (https://github.com/qmlvietnam/CodeforBlog/blob/main/QPE%20(1).ipynb)
    cho ma trận đơn nhất U bất kỳ.
    '''
    
    def __init__(self,
                 U,
                 input_qubits,
                 num_output_qubits=None,
                 output_qubits=None, initial_circuit=[],measure_or_sim=False):


        self.U = U
        self.input_qubits = input_qubits
        self.num_input_qubits = len(self.input_qubits)
        self.initial_circuit = initial_circuit
        self.measure_or_sim = measure_or_sim


        if output_qubits is not None:
            self.output_qubits = output_qubits
            self.num_output_qubits = len(self.output_qubits)
            
        elif num_output_qubits is not None:
            self.num_output_qubits = num_output_qubits
            self.output_qubits = [cirq.LineQubit(i) for i 
               in range(self.num_input_qubits,self.num_input_qubits+self.num_output_qubits)]
            
        else:
            raise ValueError("Alteast one of num_output_qubits or output_qubits to be specified")
        
        self.num_qubits = self.num_input_qubits+self.num_output_qubits
    

    def inv_qft(self):
        self._qft_= QFT(qubits=self.output_qubits)
        self._qft_.qft_circuit()
        print('print',self._qft_)
        self.QFT_inv_circuit =  self._qft_.inv_circuit
        

    def circuit(self):
        self.circuit = cirq.Circuit()
        self.circuit.append(cirq.H.on_each(*self.output_qubits))
        print(self.circuit)
        print(self.output_qubits)
        print(self.input_qubits)
        print((self.output_qubits + self.input_qubits))
        self.qubits = list(self.input_qubits + self.output_qubits) 
        self.circuit.append(ControlledUnitary(self.num_qubits,
                                         self.num_input_qubits,self.U)(*self.qubits))
        self.inv_qft()
        self.circuit.append(self.QFT_inv_circuit)
        if len(self.initial_circuit) > 0 :
            self.circuit = self.initial_circuit + self.circuit
    
    def measure(self):
        self.circuit.append(cirq.measure(*self.output_qubits,key='m'))
        
        
    def simulate_circuit(self,measure=True):
        sim = cirq.Simulator()
        if measure == False:
            result = sim.simulate(self.circuit)
        else:
            result = sim.run(self.circuit, repetitions=1000).histogram(key='m')
        return 

# Eigenvalue Inversion

In [13]:
class EigenValueInversion(cirq.Gate):
    """
    Class EigenValueInversion sẽ nghịch đảo giá trị riêng. Ứng với mỗi giá trị riêng,
    ta áp dụng phép xoay R_y tương ứng.
    """

    def __init__(self, num_qubits, C, t):
        super(EigenValueInversion, self)
        self._num_qubits = num_qubits
        self.C = C
        self.t = t
        # No of possible Eigen values self.N
        self.N = 2**(num_qubits-1)

    def num_qubits(self):
        return self._num_qubits

    def _decompose_(self, qubits):
        """
        Apply the Rotation Gate for each possible 
        # Eigen value corresponding to the Eigen
        # value basis state. For each input basis state 
        # only the Rotation gate corresponding to it would be 
        # applied to the ancilla qubit
        """
        base_state = 2**self.N - 1

        for eig_val_state in range(self.N):
            eig_val_gate = self._ancilla_rotation(eig_val_state)

            if (eig_val_state != 0):
                base_state = eig_val_state - 1
            # XOR between successive eigen value states to 
            # determine the qubits  to flip
            qubits_to_flip = eig_val_state ^ base_state

            # Apply the flips to the qubits as determined 
            # by the XOR operation 

            for q in qubits[-2::-1]:

                if qubits_to_flip % 2 == 1:
                    yield cirq.X(q)
                qubits_to_flip >>= 1

                # Build controlled ancilla rotation
                eig_val_gate = cirq.ControlledGate(eig_val_gate)
            # Controlled Rotation Gate with the 1st (num_qubits -1) qubits as
            # control qubit and the last qubit as the target qubit(ancilla)

            yield eig_val_gate(*qubits)

    def _ancilla_rotation(self, eig_val_state):
        if eig_val_state == 0:
            eig_val_state = self.N
        theta = 2*math.asin(self.C * self.N * self.t / (2*np.pi * eig_val_state))
        # Rotation around the y axis by angle theta 
        return cirq.ry(theta)

# HHL Algorithm

In [16]:
class HHL:

    def __init__(self, hamiltonian, initial_state=None, initial_state_transforms=None, qpe_register_size=4, C=None, t=1):
        """
        :param hamiltonian: ma trận Hermitan đầu vào 
        :param C: C
        :param t: t
        :param initial_state: véc-tơ |b> đầu vào
        """
        self.hamiltonian = hamiltonian
        self.initial_state = initial_state
        self.initial_state_transforms = initial_state_transforms
        self.qpe_register_size = qpe_register_size # số qubit của thanh ghi số 2 cho thuật toán QPE
        self.C = C
        self.t = t

        const = self.t/np.pi
        self.t = const*np.pi
        if self.C is None:
            self.C = 2*np.pi / (2**self.qpe_register_size * t)

    def build_hhl_circuit(self):
        # Khởi tạo cấu trúc mạch cho thuật toán QFT. 
        # Về sau các phép biến đổi sẽ được thêm vào bằng circuit.append()
        self.circuit = cirq.Circuit()
        # Khởi tạo Ancilla qubit = |0> cho thanh ghi thứ nhất
        self.ancilla_qubit = cirq.LineQubit(0)
        # Khởi tạo các qubits của thanh ghi thứ 2 có giá trị |0>
        self.qpe_register = [cirq.LineQubit(i) for i in range(1, self.qpe_register_size+1)]

        # khởi tạo các qubits mã hóa véc-tơ |b>. Nếu initial_state_transforms = None,
        # véc-tơ |b> được cho giá trị là |0>
        if self.initial_state is None:
            self.initial_state_size = int(np.log2(self.hamiltonian.shape[0]))
            if self.initial_state_size == 1:
                self.initial_state = [cirq.LineQubit(self.qpe_register_size + 1)]
            else:
                self.initial_state = [cirq.LineQubit(i) for i in range(self.qpe_register_size + 1,
                                               self.qpe_register_size + 1 + self.initial_state_size)]
        for op in list(self.initial_state_transforms):
            print(op)
            self.circuit.append(op(self.initial_state[0]))

        # Tạo ma trận đơn nhất
        self.U = HamiltonianSimulation(_H_=self.hamiltonian, t=self.t)
        # Quantum Phase Estimation của ma trận U và véc-tơ |b>
        _qpe_ = QuantumPhaseEstimation(input_qubits=self.initial_state,
                                       output_qubits=self.qpe_register, U=self.U)
        _qpe_.circuit()
        print(dir(_qpe_))
        print('CIRCUIT',_qpe_.circuit)
        self.circuit += _qpe_.circuit
        # Nghịch đảo giá trị riêng bằng hàm EigenValueInversion
        _eig_val_inv_ = EigenValueInversion(num_qubits=self.qpe_register_size + 1, C=self.C, t=self.t)
        self.circuit.append(_eig_val_inv_(*(self.qpe_register + [self.ancilla_qubit])))
        
        print(self.circuit)
        # Nghịch đảo các phép toán của QPE
        self.circuit.append(_qpe_.circuit**(-1))
        # Thực hiện phép đo trên ancilla qubit
        self.circuit.append(cirq.measure(self.ancilla_qubit,key='a'))
        # Thêm các observables để tính giá trị kỳ vọng
        self.circuit.append([
            cirq.PhasedXPowGate(
                exponent=sympy.Symbol('exponent'),
                phase_exponent=sympy.Symbol('phase_exponent'))(*self.initial_state),
            cirq.measure(*self.initial_state, key='m')
        ])

    def simulate(self):
      # # Mô phỏng chương trình trên máy tính lượng tử và in ra kết quả
        simulator = cirq.Simulator()

        # Khởi tạo các observables: Pauli-X, Pauli-Y, và Pauli-Z 
        params = [{
            'exponent': 0.5,
            'phase_exponent': -0.5
        }, {
            'exponent': 0.5,
            'phase_exponent': 0
        }, {
            'exponent': 0,
            'phase_exponent': 0
        }]

        results = simulator.run_sweep(self.circuit, params, repetitions=5000)

        for label, result in zip(('X', 'Y', 'Z'), list(results)):
            expectation = 1 - 2 * np.mean(
                result.measurements['m'][result.measurements['a'] == 1])
            print('{} = {}'.format(label, expectation))

# Simulation

In [17]:
A = np.array([[4.30213466 - 6.01593490e-08j,
                   0.23531802 + 9.34386156e-01j],
                  [0.23531882 - 9.34388383e-01j,
                   0.58386534 + 6.01593489e-08j]])
t = 0.358166 * np.pi
C = None
qpe_register_size = 4
initial_state_transforms = [cirq.rx(1.276359), cirq.rz(1.276359)]
true_values = (0.144130, 0.413217, -0.899154)
_hhl_ = HHL(hamiltonian=A,initial_state_transforms=initial_state_transforms,qpe_register_size=4)
_hhl_.build_hhl_circuit()
_hhl_.simulate()
print("-----True Values------")
print(true_values)

Rx(0.4062776880196569π)
Rz(0.4062776880196569π)
1: ───H───

2: ───H───

3: ───H───

4: ───H───
[cirq.LineQubit(1), cirq.LineQubit(2), cirq.LineQubit(3), cirq.LineQubit(4)]
[cirq.LineQubit(5)]
[cirq.LineQubit(1), cirq.LineQubit(2), cirq.LineQubit(3), cirq.LineQubit(4), cirq.LineQubit(5)]
Circuit after processing Qubit: 0 
1: ───H───
Circuit after processing Qubit: 1 
1: ───H───@────────────
          │
2: ───────@^-0.5───H───
Circuit after processing Qubit: 2 
                   ┌────────┐
1: ───H───@──────────@───────────────────────
          │          │
2: ───────@^-0.5────H┼──────────@────────────
                     │          │
3: ──────────────────@^-0.25────@^-0.5───H───
                   └────────┘
Circuit after processing Qubit: 3 
                   ┌────────┐   ┌──────────────┐   ┌────────┐
1: ───H───@──────────@─────────────────@─────────────────────────────────────
          │          │                 │
2: ───────@^-0.5────H┼───────────@─────┼─────────────@───────────